In [57]:
import collections
import os
import random
import urllib
import zipfile
import numpy as np
import tensorflow as tf

learning_rate  = 0.1
batch_size = 128
num_steps = 3000000
display_step = 10000
eval_step = 200000

eval_words = ['nine','of','going','hardware','american','britain']

embedding_size = 200  # 词向量纬度
max_vocabulary_size = 50000  #词库语料数
min_occurance = 10 #最小词频
skip_window = 3  #左右窗口大小
num_skips = 2  #一次制作多少输入输出对
num_negsample = 64  #负采样


In [58]:
data_path = '/Users/icake/PycharmProjects/NLP_Project/TensorFLow_Project1/Unit9_Project/text8.zip'

with zipfile.ZipFile(data_path) as f:
    text_word = f.read(f.namelist()[0]).lower().split()

len(text_word)

17005207

In [35]:
text_word

[b'anarchism',
 b'originated',
 b'as',
 b'a',
 b'term',
 b'of',
 b'abuse',
 b'first',
 b'used',
 b'against',
 b'early',
 b'working',
 b'class',
 b'radicals',
 b'including',
 b'the',
 b'diggers',
 b'of',
 b'the',
 b'english',
 b'revolution',
 b'and',
 b'the',
 b'sans',
 b'culottes',
 b'of',
 b'the',
 b'french',
 b'revolution',
 b'whilst',
 b'the',
 b'term',
 b'is',
 b'still',
 b'used',
 b'in',
 b'a',
 b'pejorative',
 b'way',
 b'to',
 b'describe',
 b'any',
 b'act',
 b'that',
 b'used',
 b'violent',
 b'means',
 b'to',
 b'destroy',
 b'the',
 b'organization',
 b'of',
 b'society',
 b'it',
 b'has',
 b'also',
 b'been',
 b'taken',
 b'up',
 b'as',
 b'a',
 b'positive',
 b'label',
 b'by',
 b'self',
 b'defined',
 b'anarchists',
 b'the',
 b'word',
 b'anarchism',
 b'is',
 b'derived',
 b'from',
 b'the',
 b'greek',
 b'without',
 b'archons',
 b'ruler',
 b'chief',
 b'king',
 b'anarchism',
 b'as',
 b'a',
 b'political',
 b'philosophy',
 b'is',
 b'the',
 b'belief',
 b'that',
 b'rulers',
 b'are',
 b'unnecessa

In [59]:
# 计数器
# 计算每个词出现了多少次
count = [('UNK',-1)]

#基于词频 返回max_vocabulary_size（词库语料数）个常用词
count.extend(collections.Counter(text_word).most_common(max_vocabulary_size -1 ))


In [37]:
count[:10]

[('UNK', -1),
 (b'the', 1061396),
 (b'of', 593677),
 (b'and', 416629),
 (b'one', 411764),
 (b'in', 372201),
 (b'a', 325873),
 (b'to', 316376),
 (b'zero', 264975),
 (b'nine', 250430)]

In [60]:
for i in range(len(count)-1,-1,-1):
    if count[i][1] < min_occurance:
        count.pop(i)
    else:
        break

In [61]:
vocabulary_size = len(count)
word2id = dict()
for i,(word,_) in enumerate(count):
    word2id[word] = i

In [40]:
word2id

{'UNK': 0,
 b'the': 1,
 b'of': 2,
 b'and': 3,
 b'one': 4,
 b'in': 5,
 b'a': 6,
 b'to': 7,
 b'zero': 8,
 b'nine': 9,
 b'two': 10,
 b'is': 11,
 b'as': 12,
 b'eight': 13,
 b'for': 14,
 b's': 15,
 b'five': 16,
 b'three': 17,
 b'was': 18,
 b'by': 19,
 b'that': 20,
 b'four': 21,
 b'six': 22,
 b'seven': 23,
 b'with': 24,
 b'on': 25,
 b'are': 26,
 b'it': 27,
 b'from': 28,
 b'or': 29,
 b'his': 30,
 b'an': 31,
 b'be': 32,
 b'this': 33,
 b'which': 34,
 b'at': 35,
 b'he': 36,
 b'also': 37,
 b'not': 38,
 b'have': 39,
 b'were': 40,
 b'has': 41,
 b'but': 42,
 b'other': 43,
 b'their': 44,
 b'its': 45,
 b'first': 46,
 b'they': 47,
 b'some': 48,
 b'had': 49,
 b'all': 50,
 b'more': 51,
 b'most': 52,
 b'can': 53,
 b'been': 54,
 b'such': 55,
 b'many': 56,
 b'who': 57,
 b'new': 58,
 b'used': 59,
 b'there': 60,
 b'after': 61,
 b'when': 62,
 b'into': 63,
 b'american': 64,
 b'time': 65,
 b'these': 66,
 b'only': 67,
 b'see': 68,
 b'may': 69,
 b'than': 70,
 b'world': 71,
 b'i': 72,
 b'b': 73,
 b'would': 74,
 b'd

In [62]:
data = list()
unk_count = 0
for word in text_word:
    index = word2id.get(word,0)
    if index == 0:
        unk_count += 1
    data.append(index)

count[0] = ('UNK',unk_count)
id2word = dict(zip(word2id.values(),word2id.keys()))


print('总词数（含重复）',len(text_word))
print('总词数（不重复）',len(set(text_word)))
print('语料库大小',vocabulary_size)
print('最常出现的词',count[:10])

总词数（含重复） 17005207
总词数（不重复） 253854
语料库大小 47135
最常出现的词 [('UNK', 444176), (b'the', 1061396), (b'of', 593677), (b'and', 416629), (b'one', 411764), (b'in', 372201), (b'a', 325873), (b'to', 316376), (b'zero', 264975), (b'nine', 250430)]


In [50]:
print('总词数（含重复）',len(text_word))
print('总词数（不重复）',len(set(text_word)))
print('语料库大小',vocabulary_size)
print('最常出现的词',count[:10])

总词数（含重复） 17005207
总词数（不重复） 253854
语料库大小 47135
最常出现的词 [('UNK', 444176), (b'the', 1061396), (b'of', 593677), (b'and', 416629), (b'one', 411764), (b'in', 372201), (b'a', 325873), (b'to', 316376), (b'zero', 264975), (b'nine', 250430)]


In [63]:
data_index = 0
def next_batch(batch_size,num_skips,skip_window):
    global  data_index
    assert batch_size %num_skips == 0
    assert num_skips <= 2*skip_window

    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size,1),dtype=np.int32)

    span = 2*skip_window +1
    buffer = collections.deque(maxlen=span)   # 设置一个长度为7的队列
    if data_index+span > len(data):       #如果数据被滑完了
        data_index = 0
    buffer.extend(data[data_index:data_index+span])   # 队列里存的是当前的数据
    data_index += span
    for i in range(batch_size // num_skips):
        context_word = [w for w in range(span) if w != skip_window]
        words_to_use = random.sample(context_word,num_skips)
        for j,context_word in enumerate (words_to_use):
            batch[ i*num_skips + j ] = buffer [skip_window]
            labels[ i*num_skips + j ] = buffer [context_word]
        if data_index == len(data):
            buffer.extend(data[0:span])
            data_index = span
        else:
            buffer.append(data[data_index])
            data_index +=1

    data_index = (data_index+ len(data) - span) % len(data)
    return batch,labels

In [64]:
with tf.device('/cpu:0'):
    embedding = tf.Variable(tf.random.normal([vocabulary_size,embedding_size]))
    nce_weight = tf.Variable(tf.random.normal([vocabulary_size,embedding_size]))
    nce_biases = tf.Variable(tf.zeros(vocabulary_size))

def get_embedding(x):
    with tf.device('/cpu:0'):
        x_embed = tf.nn.embedding_lookup(embedding,x)
        return x_embed

In [65]:
def nce_loss(x_embed, y):
    with tf.device('/cpu:0'):
        y = tf.cast(y,tf.int64)
        loss = tf.reduce_mean(
            tf.nn.nce_loss(weights=nce_weight,
                           biases=nce_biases,
                           labels=y,
                           inputs=x_embed,
                           num_sampled=num_negsample,
                           num_classes=vocabulary_size)
        )
        return loss

In [66]:
def evaluate(x_embed):
    with tf.device('/cpu:0'):
        x_embed = tf.cast(x_embed,tf.float32)
        x_embed_norm = x_embed/ tf.sqrt(tf.reduce_sum(tf.square(x_embed)))
        embedding_norm = embedding / tf.sqrt(tf.reduce_sum(tf.square(embedding)))
        cosine_sim_op = tf.matmul(x_embed_norm,embedding_norm,transpose_b=True)
        return cosine_sim_op
'''
优化器
'''
optimizer = tf.optimizers.legacy.SGD(learning_rate=learning_rate)

In [67]:
def run_optimizer(x,y):
    with tf.device('/cpu:0'):
        with tf.GradientTape()  as g:
            emb = get_embedding(x)
            loss = nce_loss(emb,y)

        # 计算梯度
        gradients = g.gradient(loss,[embedding,nce_weight,nce_biases])

        # 更新
        optimizer.apply_gradients(zip(gradients,[embedding,nce_weight,nce_biases]))

In [68]:
x_test = np.array([word2id[w.encode('utf-8')] for w in eval_words])

for step in range(1,num_steps +1 ):
    batct_x,batch_y = next_batch(batch_size,num_skips,skip_window)
    run_optimizer(batct_x,batch_y)

    if step % display_step == 0 or step == 1:
        loss = nce_loss(get_embedding(batct_x),batch_y)
        print('step: %i, loss: % f' % (step,loss))

    if step % eval_step == 0 or step == 1:
        print('eva')
        sim = evaluate(get_embedding(x_test)).numpy()
        for i in range(len(eval_words)):
            top_k = 8
            nearest = (-sim[i,:]).argsort()[1:top_k+1]
            log_str = '%s nearest neighbor:' % eval_words[i]
            for k in range(top_k):
                log_str = '%s %s' % (log_str,id2word[nearest[k]])
            print(log_str)

step: 1, loss:  529.777649
eva
nine nearest neighbor: b'kelvin' b'waters' b'oecs' b'correctly' b'seo' b'crossings' b'cleopatra' b'autopsy'
of nearest neighbor: b'urinary' b'lobo' b'tokyo' b'accuse' b'satanic' b'aspect' b'fari' b'corpus'
going nearest neighbor: b'racist' b'plugins' b'resign' b'plentiful' b'navigate' b'hodge' b'krypton' b'dharma'
hardware nearest neighbor: b'distinctly' b'ledge' b'bethe' b'garrick' b'charts' b'neonatal' b'mondegreen' b'assigning'
american nearest neighbor: b'mancha' b'barnes' b'beltane' b'pop' b'va' b'litovsk' b'gdp' b'homelessness'
britain nearest neighbor: b'rutger' b'existence' b'endangering' b'zeitgeist' b'psp' b'cardiff' b'sampa' b'haman'
step: 10000, loss:  90.492905
step: 20000, loss:  78.218040
step: 30000, loss:  41.386723
step: 40000, loss:  27.095304
step: 50000, loss:  42.450005
step: 60000, loss:  34.945866


KeyboardInterrupt: 